In [1]:
import os
from tqdm import tqdm
from pathlib import Path
import napari
import czifile
import numpy as np
from skimage import exposure
from skimage.measure import regionprops
import pandas as pd
import pyclesperanto_prototype as cle
import plotly.express as px

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [4]:
# Read the image file and remove singleton dimensions
img = czifile.imread("./raw_data/Image 2.czi")
img = img.squeeze()

# Slice img to extract the fluorescence channel and discard brightfield data
fluo_img = img[:, 0, :, :]

# Display in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_image(fluo_img)

<Image layer 'fluo_img' at 0x20ba15883d0>

In [5]:
# Extract the first dimension size
num_slices = fluo_img.shape[0]

# Loop over all slices in the multiposition stack
for i in range(num_slices):

    # Extract each slice position according to the loop index (i)
    slice = fluo_img[i, :, :]

    # Add input slice to Napari
    viewer.add_image(slice)

    # Apply Contrast Stretching to improve segmentation accuracy
    # p2, p98 = np.percentile(slice, (2, 98)) 

    # contrast_stretched =  exposure.rescale_intensity(slice, in_range=(p2, p98))

    # Apply Voronoi-Otsu labeling to segment each cell entity
    segmented = cle.voronoi_otsu_labeling(slice, spot_sigma=1, outline_sigma=1)
    segmented = cle.pull(segmented) # Transform OCL_array into numpy for regionprops

    # Add resulting labels to Napari 
    viewer.add_labels(segmented)

    # Measure fluorescence intensity in each of the labels
    props = regionprops(label_image=segmented, intensity_image=slice)

    print(f"Image 2 #{i+1}")
    print(props[0].intensity_mean)

Image 2 #1
152.4070796460177
Image 2 #2
127.44635193133047
Image 2 #3
129.7325581395349
Image 2 #4
97.1989247311828
Image 2 #5
36.4
Image 2 #6
29.4375
Image 2 #7
14.5
Image 2 #8
45.08108108108108
Image 2 #9
36.01851851851852
Image 2 #10
23.964285714285715
